In [1]:
import folium as fm
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
from fiona.drvsupport import supported_drivers
import panel as pn
import re
from fastkml.kml import KML
import matplotlib.pyplot as plt
pn.extension(sizing_mode="stretch_width")

In [2]:
# If required

#import os
#os.getcwd()
#os.chdir()

In [2]:
supported_drivers['KML'] = 'rw'
project_zips = gpd.read_file('./Data/Geographies/DC_MD_VA_Zipcodes.kml', driver='KML')

In [3]:
# Checkpoint, it should be 1432 entries
project_zips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         1432 non-null   object  
 1   Description  1432 non-null   object  
 2   geometry     1432 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 33.7+ KB


In [3]:
def find_zip_code(cell):
    match = re.match(r'<at><openparen>([0-9]+)<closeparen>', cell)
    if match:
        return match.group(1)
zips = list()
for _,r in project_zips.iterrows():
    zips.append(find_zip_code(r['Name']))
project_zips['Zip Code'] = zips
project_zips.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Name         1432 non-null   object  
 1   Description  1432 non-null   object  
 2   geometry     1432 non-null   geometry
 3   Zip Code     1432 non-null   object  
dtypes: geometry(1), object(3)
memory usage: 44.9+ KB


In [4]:
NCRbreweries = pd.read_csv('Data/NCRbreweries.csv')

In [8]:
# Check to ensure it pulls in correctly, should be 379 entries
NCRbreweries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       379 non-null    int64  
 1   obdb_id          379 non-null    object 
 2   name             379 non-null    object 
 3   brewery_type     379 non-null    object 
 4   street           324 non-null    object 
 5   address_2        0 non-null      float64
 6   address_3        0 non-null      float64
 7   city             379 non-null    object 
 8   state            379 non-null    object 
 9   county_province  0 non-null      float64
 10  postal_code      379 non-null    object 
 11  website_url      318 non-null    object 
 12  phone            337 non-null    float64
 13  country          379 non-null    object 
 14  longitude        193 non-null    float64
 15  latitude         193 non-null    float64
 16  tags             0 non-null      float64
dtypes: float64(7), i

In [5]:
# Need to change column name in NCR breweries in order to merge the datasets
NCRbreweries = NCRbreweries.rename(columns={'postal_code' : 'Zip Code'})

In [9]:
NCRbreweries.head()

,Unnamed: 0,obdb_id,name,brewery_type,street,address_2,address_3,city,state,county_province,Zip Code,website_url,phone,country,longitude,latitude,tags
0,1,1623-brewing-co-llc-westminister,"1623 Brewing CO, llc",contract,1146 colonel Joshua Ct,NaN,NaN,Westminister,Maryland,NaN,21157,NaN,NaN,United States,NaN,NaN,NaN
1,2,1781-brewing-company-spotsylvania,1781 Brewing Company,micro,11109 Plank Rd,NaN,NaN,Spotsylvania,Virginia,NaN,22553-4258,NaN,5.408413e+09,United States,NaN,NaN,NaN
2,3,2-silos-brewing-company-manassas,2 Silos Brewing Company,micro,9925 Discovery Blvd,NaN,NaN,Manassas,Virginia,NaN,20109,http://www.2silosbrewing.com,7.034202e+09,United States,NaN,NaN,NaN
3,4,2-witches-winery-and-brewing-company-danville,2 Witches Winery and Brewing Company,micro,209 Trade St,NaN,NaN,Danville,Virginia,NaN,24541-3545,http://www.2witcheswinebrew.com,4.345493e+09,United States,-79.421256,36.585903,NaN
4,5,3-stars-brewing-co-washington,3 Stars Brewing Co,micro,6400 Chillum Pl NW Ste B,NaN,NaN,Washington,District of Columbia,NaN,20012-2111,http://www.3starsbrewing.com,2.026700e+09,United States,NaN,NaN,NaN


In [6]:
combinedrecreation_df = pd.merge(project_zips, NCRbreweries, on = 'Zip Code', how = 'left')

In [7]:
combinedrecreation_df.head()

,Name,Description,geometry,Zip Code,Unnamed: 0,obdb_id,name,brewery_type,street,address_2,address_3,city,state,county_province,website_url,phone,country,longitude,latitude,tags
0,<at><openparen>20001<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.02758 38.90964 0.00000, -77.02...",20001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,<at><openparen>20002<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.01217 38.89209 0.00000, -77.01...",20002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,<at><openparen>20003<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.01402 38.88236 0.00000, -77.01...",20003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<at><openparen>20004<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.03365 38.89735 0.00000, -77.03...",20004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,<at><openparen>20005<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.03654 38.90252 0.00000, -77.03...",20005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Checkpoint: merged dataset should have 1437 entries

combinedrecreation_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1437 entries, 0 to 1436
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Name             1437 non-null   object  
 1   Description      1437 non-null   object  
 2   geometry         1437 non-null   geometry
 3   Zip Code         1437 non-null   object  
 4   Unnamed: 0       58 non-null     float64 
 5   obdb_id          58 non-null     object  
 6   name             58 non-null     object  
 7   brewery_type     58 non-null     object  
 8   street           43 non-null     object  
 9   address_2        0 non-null      float64 
 10  address_3        0 non-null      float64 
 11  city             58 non-null     object  
 12  state            58 non-null     object  
 13  county_province  0 non-null      float64 
 14  website_url      43 non-null     object  
 15  phone            46 non-null     float64 
 16  country          58 non-null     o

In [10]:
nonnullcombinedrecreation_df = combinedrecreation_df[combinedrecreation_df['name'].notnull()]

In [11]:
nonnullcombinedrecreation_df.head()

,Name,Description,geometry,Zip Code,Unnamed: 0,obdb_id,name,brewery_type,street,address_2,address_3,city,state,county_province,website_url,phone,country,longitude,latitude,tags
80,<at><openparen>20629<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.46425 38.34189 0.00000, -76.46...",20629,293.0,ruddy-duck-brewery-and-grill-dowell,Ruddy Duck Brewery and Grill,brewpub,13200 Dowell Rd,NaN,NaN,Dowell,Maryland,NaN,http://www.ruddyduckbrewery.com,4.103944e+09,United States,-76.457950,38.350501,NaN
86,<at><openparen>20639<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.68553 38.63222 0.00000, -76.68...",20639,171.0,gypsy-brewing-company-huntingtown,Gypsy Brewing Company,micro,NaN,NaN,NaN,Huntingtown,Maryland,NaN,NaN,NaN,United States,NaN,NaN,NaN
137,<at><openparen>20736<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.70067 38.67013 0.00000, -76.70...",20736,296.0,scorpion-brewing-owings,Scorpion Brewing,micro,"929 Skinners Turn Rd, Suite 100",NaN,NaN,Owings,Maryland,NaN,http://www.scorpionbrewing.com,7.033072e+09,United States,NaN,NaN,NaN
161,<at><openparen>20776<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.69655 38.84716 0.00000, -76.69...",20776,188.0,hopscratch-farm-and-brewery-harwood,HopScratch Farm & Brewery,planning,NaN,NaN,NaN,Harwood,Maryland,NaN,NaN,NaN,United States,NaN,NaN,NaN
199,<at><openparen>20877<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.21685 39.14884 0.00000, -77.21...",20877,164.0,greene-growlers-gaithersburg,Greene Growlers,brewpub,227 E Diamond Ave,NaN,NaN,Gaithersburg,Maryland,NaN,http://www.greenegrowlers.com,2.402616e+09,United States,-77.193556,39.141936,NaN


In [14]:
nonnullcombinedrecreation_df['count'] = 1

/home/sam/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [15]:
nonnullcombinedrecreation_df

,Name,Description,geometry,Zip Code,Unnamed: 0,obdb_id,name,brewery_type,street,address_2,...,city,state,county_province,website_url,phone,country,longitude,latitude,tags,count
80,<at><openparen>20629<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.46425 38.34189 0.00000, -76.46...",20629,293.0,ruddy-duck-brewery-and-grill-dowell,Ruddy Duck Brewery and Grill,brewpub,13200 Dowell Rd,NaN,...,Dowell,Maryland,NaN,http://www.ruddyduckbrewery.com,4.103944e+09,United States,-76.457950,38.350501,NaN,1
86,<at><openparen>20639<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.68553 38.63222 0.00000, -76.68...",20639,171.0,gypsy-brewing-company-huntingtown,Gypsy Brewing Company,micro,NaN,NaN,...,Huntingtown,Maryland,NaN,NaN,NaN,United States,NaN,NaN,NaN,1
137,<at><openparen>20736<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.70067 38.67013 0.00000, -76.70...",20736,296.0,scorpion-brewing-owings,Scorpion Brewing,micro,"929 Skinners Turn Rd, Suite 100",NaN,...,Owings,Maryland,NaN,http://www.scorpionbrewing.com,7.033072e+09,United States,NaN,NaN,NaN,1
161,<at><openparen>20776<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.69655 38.84716 0.00000, -76.69...",20776,188.0,hopscratch-farm-and-brewery-harwood,HopScratch Farm & Brewery,planning,NaN,NaN,...,Harwood,Maryland,NaN,NaN,NaN,United States,NaN,NaN,NaN,1
199,<at><openparen>20877<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.21685 39.14884 0.00000, -77.21...",20877,164.0,greene-growlers-gaithersburg,Greene Growlers,brewpub,227 E Diamond Ave,NaN,...,Gaithersburg,Maryland,NaN,http://www.greenegrowlers.com,2.402616e+09,United States,-77.193556,39.141936,NaN,1
203,<at><openparen>20882<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.21564 39.19262 0.00000, -77.21...",20882,129.0,elder-pine-brewing-and-blending-co-gaithersburg,Elder Pine Brewing and Blending Co,micro,4200 Sundown Rd,NaN,...,Gaithersburg,Maryland,NaN,http://www.elderpine.com,NaN,United States,NaN,NaN,NaN,1
272,<at><openparen>21113<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.75126 39.03471 0.00000, -76.74...",21113,108.0,crooked-crab-brewing-company-odenton,Crooked Crab Brewing Company,micro,8251 Telegraph Rd,NaN,...,Odenton,Maryland,NaN,http://www.cookedcrabbrewing.com,4.435699e+09,United States,-76.694824,39.097292,NaN,1
273,<at><openparen>21114<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.70788 39.01889 0.00000, -76.70...",21114,99.0,chesepiooc-real-ale-brewery-crofton,Chesepiooc Real Ale Brewery,micro,2408 Crofton Blvd,NaN,...,Crofton,Maryland,NaN,http://www.brewcrab.com,4.106302e+09,United States,-76.698047,39.016872,NaN,1
291,<at><openparen>21157<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-77.09262 39.45952 0.00000, -77.09...",21157,1.0,1623-brewing-co-llc-westminister,"1623 Brewing CO, llc",contract,1146 colonel Joshua Ct,NaN,...,Westminister,Maryland,NaN,NaN,NaN,United States,NaN,NaN,NaN,1
297,<at><openparen>21201<closeparen>,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((-76.63413 39.29435 0.00000, -76.63...",21201,114.0,dempseys-brew-pub-and-restaurant-baltimore-bal...,Dempsey's Brew Pub and Restaurant - Baltimore,brewpub,333 W Camden St,NaN,...,Baltimore,Maryland,NaN,http://www.dempseysbaltimore.com,4.108438e+09,United States,-76.620138,39.285185,NaN,1
